### Introduction to Semantic Kernel

In this lab, we will learn how to use Semantic Kernel to interact with Large Language Models. We'll complete the following steps:

1) Import general dependencies.
2) Instantiate the Kernel.
3) Connect the Kernel to a Completion model and run a simple prompt.
4) Connect the Kernel to a Chat Completion model and run a simple prompt.

#### Step 1 - Import Dependecies

**Semantic Kernel** is a SDK currently available in C#, Java and Python. However, each language support different at the current stage. Please visit [supported languages](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages) to check which features are supported per language.

In this lab, we'll use Python SDK which can be obtained in Pypi repository.

**Important:** if you get an error running the code below, make sure that you've completed the setup process. Optionally, you can run `pip install semantic_kernel` in your terminal or `!python -m pip install semantic_kernel` in a Python code cell.

In [78]:
#Importing the Semantic Kernel SDK that will be used in this lab. 
#The other dependencies will be imported directly in the step where they are used.
import semantic_kernel as sk

#### Step 2 - Instantiate the Kernel
The core of Semantic Kernel is it's Kernel. It's responsible to transform an User Input (or ASK) into a response by orchestrating Plugins. 

In this lab, we'll instantiate a simple Kernel with no additonal options. Optionally, we could define Logging, Memory and the set of skill we want to use. 

In [79]:
#Instantiating the kernel
kernel = sk.Kernel()

#Example of how to instantiate the kernel with a different log level
#kernel = sk.Kernel(log=sk.Log(level=sk.LogLevel.DEBUG))

#### Step 3 - Connect the Kernel to a Text Completion model and run a simple prompt
Now that we have our kernel, we can start adding capabilities to it. 

In this lab, we'll connect to an **AI Service** and use a Text Completion model from **Azure OpenAI Service**. 
The Semantic Kernel also supports **OpenAI**. Other AI Services, like **Hugging Face** will be supported shortly. You can visit this [link](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages#ai-service-endpoints) to check the most updated list.

To establish a connection between the Kernel and a Text Completion model, we first need to import **AzureTextCompletion** module and add it to the kernel as a Text Completion Service. 

We're also using a method to automatically read sensitive parameters from a `.env` file. Therefore, please ensure your file has the following parameters:

```
AZURE_OPENAI_DEPLOYMENT_NAME="NAME OF YOUR MODEL DEPLOYMENT"
AZURE_OPENAI_ENDPOINT="https://<YOUR_ENDPOINT>.openai.azure.com/"
AZURE_OPENAI_API_KEY="KEY"
```

In [80]:
#Importing the Azure Text Completion service connector
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion

# Read the model, API key and endpoint from the .env file
deploynment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

# Add the text completion service to the kernel
kernel.add_text_completion_service("Text Completion", 
                                   AzureTextCompletion("text-davinci-003", endpoint, api_key))

Now that we have connected the kernel to a large language model from Azure OpenAI Service, we can start using it to run Text Completion operations.

In Semantic Kernel we need to define a **Semantic Function** in order to run prompts using a LLM. We'll deep-dive on Semantic Functions later, so let's just focus on the basics in this step of the lab.

You can add a **Semantic Function** to the Kernel based on a **Prompt**. This is simple a textual description, or instruction, of what we want the function to do. Optionally, we can add parameters, like `{{$input}}`, to inject values in runtime.

In this example, we'll create a function to classify a Telecom problem in different categories, like Mobile Internet, Fixed Internet, etc. As **input**, we'll use the problem description. Please note that as we define the function we can also control the behavior of how we expect the kernel to run it, for example, controlling the max number of tokens in the response.

In [81]:
# Set the prompt for classifying the problem description
prompt = """{{$input}}
Classify the problem description above as a [Fixed Internet] or [Mobile Internet] category.
Only write the category with no extra text.
"""

# Create a semantic function for classifying the problem description
classify_call_function = kernel.create_semantic_function(prompt, 
                                                         max_tokens=10,
                                                         temperature=0.5)

Now that our Kernel has a new function to classify problems, we'll test it based on a problem description. We expected as a result one of the classifications previously defined in the prompt.

In [84]:
problem_description = "My wi-fi connection is terrible at my bedroom."

# Call the classify function with a problem description
classification = classify_call_function(problem_description)

# Print the result
print(f"Result: {classification}")

Result: 
Fixed Internet


#### Step 4 - Connect the Kernel to a Chat Completion model and run a simple prompt

